In [1]:
import re
from tqdm import tqdm
import numpy as npd
import pandas as pd
from math import ceil

In [2]:
DF = pd.read_csv('data/data_lower.csv', sep='\t', encoding = 'utf-8')
DF = DF[DF.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)]

In [3]:
#создадим список классов по частотности
df_classes = DF[['classtag']].copy()
df_classes['total_count'] = [1] * df_classes.shape[0]
df_classes = df_classes.groupby(['classtag'], sort=True).aggregate(sum) #создаем таблицу класс * количество парадигм
df_classes = df_classes.sort_values(by=['total_count'], ascending=False) #сортируем от большего к меньшему
df_classes = df_classes.reset_index(level=['classtag'])
cl_list = list(df_classes['classtag']) # сам список классов по убыванию
print('всего классов',df_classes.shape[0])
print(df_classes[:5])


всего классов 259
  classtag  total_count
0      м1а         7647
1      ж8а         3747
2      с7а         3625
3     ж3*а         3606
4     мо1а         2891


In [7]:
tail_df = df_classes.loc[df_classes['total_count'] <= 4] # "хвост" из маленьких классов, который просто выучим
print('классов, в которых не больше 4 слов:', tail_df.loc[tail_df['total_count'] <= 4].shape[0])
print('парадигм в этих классах:', tail_df['total_count'].sum())


классов, в которых не больше 4 слов: 153
парадигм в этих классах: 265


In [8]:
# short format to long format
def short_to_long(df):
    print('\n')
    print('short format dataframe with {} lemmas'.format(df.shape[0]))
    #print(df.iloc[0:5,0:5])
    df_long = pd.wide_to_long(df, stubnames = 'form', i = ['lemma','gender'], j = 'formtag')
    df_long = df_long.sample(frac=1, random_state=15) #перемешиваем строки
    print('made long format dataframe with {} wordforms'.format(df_long.shape[0]))
    #print(df_long[:5])
    return df_long

In [9]:
# train test split 
def train_and_test(classes):
    dataset = list(zip([], [], [], [], [], [], [], [], [], [], [], [], [], [], []))
    train = pd.DataFrame(data=dataset, columns=['lemma', 'gender', 'classtag',  
                                                "form1N","form1G","form1D","form1A","form1I","form1L",
                                                "form2N","form2G","form2D","form2A","form2I","form2L"])
    test = train.copy()
    for classtag in classes:
        #print('класс:', classtag)
        paradigms = DF.loc[DF['classtag'] == classtag]
        paradigms = paradigms.sample(frac=1, random_state=12345) #перемешиваем строки
        n = paradigms.shape[0]
        #print('всего парадигм:', n)
        n_train = ceil(n*0.75)
        train_df,test_df = paradigms[:n_train], paradigms[n_train:]
        #print('train: + {} парадигм'.format(train_df.shape[0]))
        train = train.append(train_df)
        #print('test: + {} парадигм\n'.format(test_df.shape[0]))
        test = test.append(test_df)
    train = short_to_long(train)
    test = short_to_long(test)
    return train,test
            

### создаем 2 файла для первого эксперимента: train_full.csv (3/4 от всех классов), test_full.csv для обучения самой полной модели, которую сравним с pymorphy2

In [10]:
%%time
#для каждого класса делим парадигмы на 0.75 трейн и 0.25 тест (классы из хвоста все уходят в трейн)
train, test = train_and_test(cl_list)
test.to_csv('data/test/test_full.csv', sep='\t', encoding = 'utf-8')
train.to_csv('data/train/train_full.csv', sep='\t', encoding = 'utf-8')




short format dataframe with 31961 lemmas
made long format dataframe with 383532 wordforms


short format dataframe with 10533 lemmas
made long format dataframe with 126396 wordforms
Wall time: 11.2 s
